In [1]:
from google.colab import drive
drive.mount("/content/drive")

PROJECT_ROOT = "/content/drive/MyDrive/stress-heart-ml-wesad"


Mounted at /content/drive


In [2]:
import os

PROJECT_ROOT = "/content/stress-heart-ml-wesad"
DATA_DIR = f"{PROJECT_ROOT}/data/processed"
RESULTS_DIR = f"{PROJECT_ROOT}/results"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print("Project directories ready ✅")


Project directories ready ✅


In [10]:
import os

root = "/root/.cache/kagglehub/datasets"

for path, dirs, files in os.walk(root):
    if any(f.endswith(".pkl") for f in files):
        print("FOUND:", path)
        print("FILES:", files)
        break


In [12]:
import os

root = "/root/.cache/kagglehub/datasets"

for path, dirs, files in os.walk(root):
    print(path)
    break

In [3]:
def hrv_from_bvp(bvp_win, fs=64):
    peaks, _ = find_peaks(bvp_win, distance=fs*0.4, prominence=0.2)
    if len(peaks) < 3:
        return None

    rr = np.diff(peaks) / fs

    return {
        "RMSSD": np.sqrt(np.mean(np.diff(rr)**2)),
        "SDNN": np.std(rr),
        "mean_HR": np.mean(60/rr)
    }


In [5]:
import numpy as np

def sliding_windows(signal, fs, window_sec=60, step_sec=30):
    win = int(window_sec * fs)
    step = int(step_sec * fs)
    windows = []
    for start in range(0, len(signal) - win, step):
        windows.append(signal[start:start + win])
    return windows

In [8]:
signals = data['signal']['wrist']

eda = signals['EDA']

print("EDA Shape:", eda.shape)

NameError: name 'data' is not defined

In [7]:
import numpy as np
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt

eda_raw = eda

# Convert to 1D
eda_1d = np.asarray(eda_raw).squeeze()

if eda_1d.ndim == 0:
    eda_1d = np.atleast_1d(eda_1d)

mask_bad = ~np.isfinite(eda_1d)
if mask_bad.any():
    idx = np.arange(eda_1d.size)
    good = ~mask_bad
    if good.sum() < 2:
        raise ValueError("Too many NaNs/infs in EDA to interpolate.")
    eda_1d = np.interp(idx, idx[good], eda_1d[good])

eda_1d = eda_1d.astype('float64')

print("Prepared EDA shape:", eda_1d.shape, "dtype:", eda_1d.dtype)
def butter_lowpass_filter_safe(x, cutoff=0.5, fs=4, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    padlen = 3 * max(len(a), len(b))
    if len(x) <= padlen:
        win = min(5, max(1, len(x)//5))
        return np.convolve(x, np.ones(win)/win, mode='same')

    try:
        y = filtfilt(b, a, x)
        return y
    except Exception as e:
        print("filtfilt failed, falling back to moving average. Error:", str(e))
        win = min(5, max(1, len(x)//5))
        return np.convolve(x, np.ones(win)/win, mode='same')
eda_filtered = butter_lowpass_filter_safe(eda_1d, cutoff=0.5, fs=4, order=4)
plt.figure(figsize=(14,4))
plt.plot(eda_1d, alpha=0.4, label='raw (1D)')
plt.plot(eda_filtered, linewidth=1, label='filtered')
plt.legend()
plt.title("EDA: raw vs filtered")
plt.xlabel("Samples")
plt.ylabel("EDA")
plt.show()

NameError: name 'eda' is not defined

In [6]:
eda_windows = sliding_windows(eda_filtered, fs=4)
print("EDA windows:", len(eda_windows))

NameError: name 'eda_filtered' is not defined

In [4]:
bvp_windows = sliding_windows(bvp_filt, fs=64)

NameError: name 'sliding_windows' is not defined

In [ ]:
dataset = []

for eda_w, bvp_w in zip(eda_windows, bvp_windows):
    scr_count, scr_amp = eda_stress_score(eda_w)
    hrv = hrv_from_bvp(bvp_w)

    if hrv is None:
        continue

    row = {
        "SCR_count": scr_count,
        "SCR_amp": scr_amp,
        "RMSSD": hrv["RMSSD"],
        "SDNN": hrv["SDNN"],
        "mean_HR": hrv["mean_HR"]
    }
    dataset.append(row)

import pandas as pd
df = pd.DataFrame(dataset)
df.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.scatter(df["SCR_count"], df["mean_HR"], alpha=0.6)
plt.xlabel("SCR Count (Stress Level)")
plt.ylabel("Mean Heart Rate (BPM)")
plt.title("Stress vs Heart Rate")
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(df["SCR_count"], df["RMSSD"], alpha=0.6)
plt.xlabel("SCR Count (Stress Level)")
plt.ylabel("RMSSD (HRV)")
plt.title("Stress vs HRV (RMSSD)")
plt.show()

In [ ]:
plt.figure(figsize=(14,4))
plt.plot(df["SCR_count"], label="Stress Level (SCR count)")
plt.xlabel("Window Index")
plt.ylabel("SCR Count")
plt.title("Stress Level Over Time")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(14,4))
plt.plot(df["mean_HR"], color='r', label="Mean Heart Rate")
plt.xlabel("Window Index")
plt.ylabel("BPM")
plt.title("Heart Rate Over Time")
plt.legend()
plt.show()

In [ ]:
np.save(f"{DATA_DIR}/bvp_filtered.npy", bvp_filt)
print("✅ Saved cleaned BVP")
